In [5]:
!pip install google-play-scraper
from google_play_scraper import app, reviews, Sort, reviews_all, exceptions
import csv
import time
import random

In [6]:
# Fungsi untuk menyimpan data ke file CSV
def save_to_csv(reviews, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=[
            'reviewId', 'userName', 'content', 'score',
            'thumbsUpCount', 'reviewCreatedVersion',
            'at', 'replyContent', 'repliedAt'
        ])
        writer.writeheader()
        for review in reviews:
            writer.writerow({
                'reviewId': review.get('reviewId'),
                'userName': review.get('userName'),
                'content': review.get('content'),
                'score': review.get('score'),
                'thumbsUpCount': review.get('thumbsUpCount'),
                'reviewCreatedVersion': review.get('reviewCreatedVersion'),
                # Convert datetime to string for CSV compatibility
                'at': review.get('at').strftime('%Y-%m-%d %H:%M:%S') if review.get('at') else '',
                # Reply content and replied date might be None
                'replyContent': (review['replyContent'] if review['replyContent'] else '') if isinstance(review, dict) else '',
                # Convert datetime to string or empty string if None
                'repliedAt': (review['repliedAt'].strftime('%Y-%m-%d %H:%M:%S') if isinstance(review['repliedAt'], type(review['at'])) else '') if isinstance(review, dict) and ('repliedAt' in review) else ''
            })

# Fungsi untuk scraping ulasan dengan penanganan error dan delay agar tidak diblokir server
def scrape_google_play_reviews(app_id, max_reviews=12000):
    try:
        reviews = reviews_all(
            app_id,
            lang='id',       # Bahasa Indonesia
            country='id',    # Negara Indonesia
            sort=Sort.MOST_RELEVANT,
            count=max_reviews,
            filter_score_with=None  # Ambil semua skor ulasan tanpa filter
        )

        save_to_csv(reviews, f'{app_id}_reviews.csv')
        print(f"Total reviews collected: {len(reviews)}")

    except exceptions.TooManyRequestsError:
        print("Terlalu banyak request. Menunggu beberapa menit sebelum mencoba lagi...")
        time.sleep(random.uniform(60, 120))  # Tunggu antara 1-2 menit lalu coba ulangi scraping lagi.
        scrape_google_play_reviews(app_id,max_reviews)

    except exceptions.RequestFailedError as e:
        print(f"Request gagal: {e}")

    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

#  Aplikasi target
app_id = "com.google.android.youtube"
scrape_google_play_reviews(app_id)

Total reviews collected: 54000
